In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

import plotly.graph_objs as go

In [2]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, h3=10, out_features=3):
        # instantiate nn.Module
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h3, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x

In [3]:
torch.manual_seed(42)
model = Model()

In [4]:
iris = datasets.load_iris()

idf = pd.DataFrame(
    data=np.c_[iris["data"], iris["target"]], columns=iris["feature_names"] + ["target"]
)

idf.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0
149,5.9,3.0,5.1,1.8,2.0


In [5]:
# train, test, split

X = idf.drop("target", axis=1)
y = idf["target"]

X = X.values
y = y.values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=41
)

# convert to tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

# probably want long tensors as the target is an integer
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# set criterion and optimizer

# cross entropy loss because this is a classification problem
crit = nn.CrossEntropyLoss()

# adam optimizer, popular for classification problems, with a learning rate of 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

epochs = 100
losses = []

In [7]:
for i in range(epochs):
    # forward and get a prediction
    y_pred = model.forward(X_train)
    # calculate loss
    loss = crit(y_pred, y_train)
    losses.append(loss)
    if i % 10 == 0:
        print(f"epoch {i} and loss is: {loss}")

    # backpropagation

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch 0 and loss is: 1.1077466011047363
epoch 10 and loss is: 0.3657478094100952
epoch 20 and loss is: 0.18298104405403137
epoch 30 and loss is: 0.06400630623102188
epoch 40 and loss is: 0.03511335328221321
epoch 50 and loss is: 0.02067386731505394
epoch 60 and loss is: 0.027184605598449707
epoch 70 and loss is: 0.018347231671214104
epoch 80 and loss is: 0.03135373815894127
epoch 90 and loss is: 0.016589965671300888


In [8]:
# plotting the loss

trace = go.Scatter(
    x=list(range(len(losses))),
    y=[loss.detach().numpy() for loss in losses],
    mode="lines",
    name="Loss",
)

data = [trace]

layout = go.Layout(
    title="Loss over Epochs", xaxis=dict(title="Epoch"), yaxis=dict(title="Loss")
)

fig = go.Figure(data=data, layout=layout)

fig.show()

In [9]:
# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print("Accuracy:", accuracy)

Accuracy: 0.9333333333333333


In [10]:
# create new input data
mystery_iris = torch.tensor([5.6, 3.7, 2.2, 0.5])
print(mystery_iris)

tensor([5.6000, 3.7000, 2.2000, 0.5000])


In [13]:
# predict mystery iris
with torch.no_grad():
    print(model(mystery_iris))

# print target for mystery iris
print(iris["target_names"][model(mystery_iris).argmax().item()])

tensor([  9.1789,   1.3979, -13.4546])
setosa


In [14]:
# save the model
torch.save(model.state_dict(), "simple_iris_model.pt")

In [ ]:
# load the model
# new_model = Model()
# new_model.load_state_dict(torch.load("simple_iris_model.pt"))